In [1]:
from RawHandler.utils import align_images, download_file_requests
from RawHandler.RawHandler import RawHandler
import os
import matplotlib.pyplot as plt

In [2]:
# Download example raw files
def download_file_if_needed(file_url, output_file):
    if not os.path.exists(output_file):
        download_file_requests(file_url, output_file)


low_iso_file_url = "https://dataverse.uclouvain.be/api/access/datafile/:persistentId?persistentId=doi:10.14428/DVN/DEQCIM/UNYG7K"
low_iso_output_file = "gtBark_GT.arw"
download_file_if_needed(low_iso_file_url, low_iso_output_file)

high_iso_file_url = "https://dataverse.uclouvain.be/api/access/datafile/:persistentId?persistentId=doi:10.14428/DVN/DEQCIM/AVO8RR"
high_iso_output_file = "gtBark_12800.arw"
download_file_if_needed(high_iso_file_url, high_iso_output_file)

In [3]:
# A simple code to align image is also provided for use in supervised learning.
# The first raw handler is the target to be used to align to, the second is aligned.
# The crop is specified by 'dims', and a starting offset can be supplied.
low_iso_rh = RawHandler("gtBark_GT.arw")
high_iso_rh = RawHandler("gtBark_12800.arw")
dims = (1500, 1500 + 200, 4500, 4500 + 200)

offset = align_images(high_iso_rh, low_iso_rh, dims, offset=(0, 0, 0, 0))
offset

AttributeError: 'BaseRawHandler' object has no attribute 'input_handler'

In [ ]:
# We can see the initial results with and without alignment

img_unaligned = (
    low_iso_rh.as_RGB_colorspace(dims=dims, colorspace="sRGB").transpose(1, 2, 0) * 3
)
img = (
    low_iso_rh.as_RGB_colorspace(dims=dims + offset, colorspace="sRGB").transpose(
        1, 2, 0
    )
    * 3
)

img_noisy = (
    high_iso_rh.as_RGB_colorspace(dims=dims, colorspace="sRGB").transpose(1, 2, 0) * 3
)

plt.subplots(2, 2, figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.imshow(img_noisy)
plt.title("High-ISO")

plt.subplot(2, 2, 2)
plt.imshow(img)
plt.title("Low-ISO")

plt.subplot(2, 2, 3)
plt.imshow(img_noisy - img_unaligned)
plt.title("Difference without alignment")

plt.subplot(2, 2, 4)
plt.imshow(img_noisy - img)
plt.title("Difference with alignment")

In [ ]:
# A basic dataset class for supervised learning is also included
from RawHandler.RawDataset import RawDataset

gt_list = ["gtBark_GT.arw"]
noisy_list = ["gtBark_12800.arw"]
file_pair_list = list(zip(noisy_list, gt_list))

dataset = RawDataset(file_pair_list, crop_size=256)

In [ ]:
noisy_rggb, noisy_rgb, gt_rgb, offset = dataset[0]